In [1]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  172.22.0.2  187.94 KiB  16      55.5%             71e62da2-1e9a-4bab-ae1d-d0e8f61e9422  rack1
UN  172.22.0.3  70.27 KiB   16      70.6%             bfbc6286-3efa-43c7-9770-8069a33a0a9f  rack1
UN  172.22.0.4  70.28 KiB   16      73.9%             f9d5afc9-45ca-4a78-a5e8-4cf93d6a6686  rack1



In [2]:
#Connect to the Cassandra cluster
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [3]:
#q1
#drop a weather keyspace if it already exists
cass.execute("drop keyspace if exists weather;")
#create a weather keyspace with 3x replication
cass.execute("create keyspace weather with replication={'class': 'SimpleStrategy', 'replication_factor': 3};")
#inside weather, create a station_record type containing two ints: tmin and tmax
cass.execute("""
CREATE TYPE
weather.station_record (
    tmin int,
    tmax int
)
""")
#inside weather, create a stations table
cass.execute("""
CREATE TABLE weather.stations (
    id text,
    name TEXT STATIC,
    date date,
    record station_record,
    state TEXT,
    PRIMARY KEY (id,date)
) WITH CLUSTERING ORDER BY (date ASC)
""")

print(cass.execute("describe table weather.stations ").one().create_statement)

CREATE TABLE weather.stations (
    id text,
    date date,
    name text static,
    state text,
    record station_record,
    PRIMARY KEY (id, date)
) WITH CLUSTERING ORDER BY (date ASC)
    AND additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND cdc = false
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND memtable = 'default'
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND extensions = {}
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair = 'BLOCKING'
    AND speculative_retry = '99p';


In [4]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-77b47c0b-71ba-4d9e-a30e-f07101cfa8cd;1.0
	confs: [default]


	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central


	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central


	found com.typesafe#config;1.4.1 in central


...

	[SUCCESSFUL ] org.scala-lang#scala-reflect;2.12.11!scala-reflect.jar (143ms)
downloading https://repo1.maven.org/maven2/com/datastax/oss/native-protocol/1.5.0/native-protocol-1.5.0.jar ...
	[SUCCESSFUL ] com.datastax.oss#native-protocol;1.5.0!native-protocol.jar(bundle) (39ms)
downloading https://repo1.maven.org/maven2/com/datastax/oss/java-driver-shaded-guava/25.1-jre-graal-sub-1/java-driver-shaded-guava-25.1-jre-graal-sub-1.jar ...
	[SUCCESSFUL ] com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1!java-driver-shaded-guava.jar (100ms)
downloading https://repo1.maven.org/maven2/com/typesafe/config/1.4.1/config-1.4.1.jar ...
	[SUCCESSFUL ] com.typesafe#config;1.4.1!config.jar(bundle) (32ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.26/slf4j-api-1.7.26.jar ...


	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.26!slf4j-api.jar (27ms)
downloading https://repo1.maven.org/maven2/io/dropwizard/metrics/metrics-core/4.1.18/metrics-core-4.1.18.jar ...
	[SUCCESSFUL ] io.dropwizard.metrics#metrics-core;4.1.18!metrics-core.jar(bundle) (28ms)
downloading https://repo1.maven.org/maven2/org/hdrhistogram/HdrHistogram/2.1.12/HdrHistogram-2.1.12.jar ...
	[SUCCESSFUL ] org.hdrhistogram#HdrHistogram;2.1.12!HdrHistogram.jar(bundle) (30ms)
downloading https://repo1.maven.org/maven2/org/reactivestreams/reactive-streams/1.0.3/reactive-streams-1.0.3.jar ...
	[SUCCESSFUL ] org.reactivestreams#reactive-streams;1.0.3!reactive-streams.jar (26ms)
downloading https://repo1.maven.org/maven2/com/github/stephenc/jcip/jcip-annotations/1.0-1/jcip-annotations-1.0-1.jar ...
	[SUCCESSFUL ] com.github.stephenc.jcip#jcip-annotations;1.0-1!jcip-annotations.jar (26ms)
downloading https://repo1.maven.org/maven2/com/github/spotbugs/spotbugs-annotations/3.1.12/spotbugs-annotations-3.1.12.jar ...


	[SUCCESSFUL ] com.datastax.oss#java-driver-query-builder;4.13.0!java-driver-query-builder.jar(bundle) (31ms)
:: resolution report :: resolve 5334ms :: artifacts dl 1313ms
	:: modules in use:
	com.datastax.oss#java-driver-core-shaded;4.13.0 from central in [default]
	com.datastax.oss#java-driver-mapper-runtime;4.13.0 from central in [default]
	com.datastax.oss#java-driver-query-builder;4.13.0 from central in [default]
	com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 from central in [default]
	com.datastax.oss#native-protocol;1.5.0 from central in [default]
	com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 from central in [default]
	com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 from central in [default]
	com.github.spotbugs#spotbugs-annotations;3.1.12 from central in [default]
	com.github.stephenc.jcip#jcip-annotations;1.0-1 from central in [default]
	com.google.code.findbugs#jsr305;3.0.2 from central in [default]
	com.thoughtworks.paranamer#parana

23/11/22 17:39:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
sc = spark.sparkContext # for interacting directly with RDDs 

In [6]:
#q2

In [7]:
df = spark.read.text("ghcnd-stations.txt")

In [8]:
pandas_df=df.toPandas()

In [9]:
pandas_df["station"] = pandas_df["value"].str[:11]

In [10]:
from pyspark.sql.functions import col, expr

In [11]:
df2 = df.withColumn("station", expr("substring(value, 0, 11)")).withColumn(
     "STATE",expr("substring(value,38,40)")).withColumn(
     "NAME",expr("substring(value, 41,71)"))
df2
df2=df2.filter(col("STATE").like("% WI %")).filter(col("station").like("%US%"))
df2.toPandas()

,value,station,STATE,NAME
0,US1WIAD0002 43.9544 -89.8096 294.4 WI ADAMS...,US1WIAD0002,WI ADAMS 0.4 E,ADAMS 0.4 E
1,US1WIAD0005 44.2053 -89.8480 305.7 WI NEKOO...,US1WIAD0005,WI NEKOOSA 8.0 SSE,NEKOOSA 8.0 SSE
2,US1WIAD0006 43.8858 -89.7259 307.8 WI GRAND...,US1WIAD0006,WI GRAND MARSH 1.0 W,GRAND MARSH 1.0 W
3,US1WIAD0008 43.8611 -89.7163 310.0 WI GRAND...,US1WIAD0008,WI GRAND MARSH 1.9 SSW,GRAND MARSH 1.9 SSW
4,US1WIAD0010 43.7864 -89.6417 293.8 WI OXFOR...,US1WIAD0010,WI OXFORD 4.0 W,OXFORD 4.0 W
...,...,...,...,...
1308,USW00094930 43.9333 -90.2667 280.1 WI VOLK ...,USW00094930,WI VOLK FLD ANG,VOLK FLD ANG
1309,USW00094940 43.9667 -90.7333 252.7 WI SPART...,USW00094940,WI SPARTA FT MCCOY,SPARTA FT MCCOY
1310,USW00094973 46.0303 -91.4425 368.8 WI HAYWA...,USW00094973,WI HAYWARD MUNI AP,HAYWARD MUNI AP
1311,USW00094985 44.6378 -90.1875 381.6 WI MARSH...,USW00094985,WI MARSHFIELD MUNI AP,MARSHFIELD MUNI AP


In [12]:
#Filter your results to the state of Wisconsin, collect the rows in your notebook so you can loop over them,
#and do an INSERT into your weather.stations table for each station ID and name.
insert_query = "INSERT INTO weather.stations (id, name) VALUES (?, ?)"
prepared_statement = cass.prepare(insert_query)

for row in df2.collect():
    station_WI = row["station"]
    name_WI = row["NAME"]
    values = (station_WI, name_WI)
    cass.execute(prepared_statement, values)


In [13]:
import pandas as pd

pd.DataFrame(cass.execute(
    """
    SELECT COUNT(*) FROM weather.stations
    """
))

,count
0,1313


In [14]:
pd.DataFrame(cass.execute(
    """
    SELECT name FROM weather.stations
    WHERE id = 'USW00014837'
    """
))

,name
0,MADISON DANE CO RGNL AP 72641


In [15]:
#q3

In [16]:
# Get the token for the station
token_df = pd.DataFrame(cass.execute("SELECT TOKEN(id) FROM weather.stations WHERE id = 'USC00470273'"))
token_df

,system_token_id
0,-9014250178872933741


In [17]:
#q4

In [18]:
import subprocess

# Get the token for the station
station_token = cass.execute("SELECT TOKEN(id) FROM weather.stations WHERE id = 'USC00470273'").one()[0]

# Run nodetool ring and get the output
output = subprocess.check_output(["nodetool", "ring"]).decode()

# Split the output into lines
lines = output.split("\n")

# Initialize the smallest token difference and the next token
smallest_diff = float("inf")
next_token = None

# Loop over the lines
for line in lines:
    # Split the line into columns
    columns = line.split()
    
    # Skip the line if it doesn't have enough columns
    if len(columns) < 8:
        continue
    
    # Try to get the token from the line
    try:
        token = int(columns[7])
    except ValueError:
        # Skip this line if the token is not an integer
        continue
    
    # Calculate the difference between the station token and this token
    diff = token - station_token
    
    # Handle the case where the ring "wraps around"
    if diff < 0:
        diff += 2**64
    
    # If this difference is smaller than the smallest difference we've seen so far,
    # update the smallest difference and the next token
    if diff < smallest_diff:
        smallest_diff = diff
        next_token = token

next_token


-8715368926633190629

In [19]:
#q5

In [20]:
import station_pb2
import station_pb2_grpc
import grpc

# Create a gRPC channel and stub
channel = grpc.insecure_channel('localhost:5440')
stub = station_pb2_grpc.StationStub(channel)

# Make a StationMax request
response = stub.StationMax(station_pb2.StationMaxRequest(id='USW00014837'))

response.max


ValueError: Protocol message StationMaxRequest has no "id" field.

In [21]:
#q6

In [22]:
df = spark.read.format("org.apache.spark.sql.cassandra") \
    .option("spark.cassandra.connection.host", "p6-db-1,p6-db-2,p6-db-3") \
    .option("keyspace", "weather") \
    .option("table", "stations") \
    .load()

# Create a temporary view named 'stations'
df.createOrReplaceTempView("stations")

# List the tables/views in the Spark catalog
spark.catalog.listTables()

[Table(name='stations', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [23]:
#q7

In [24]:
# Compute the average difference between tmax and tmin for each station
result = spark.sql("""
SELECT id, AVG(record.tmax - record.tmin) as avg_diff
FROM stations
WHERE record.tmax IS NOT NULL AND record.tmin IS NOT NULL
GROUP BY id
""")

# Convert the result to a dictionary
result_dict = {row.id: row.avg_diff for row in result.collect()}

result_dict


23/11/22 17:41:01 ERROR CassandraConnectorConf: Unknown host 'p6-db-2'
java.net.UnknownHostException: p6-db-2: Temporary failure in name resolution
	at java.net.Inet4AddressImpl.lookupAllHostAddr(Native Method)
	at java.net.InetAddress$2.lookupAllHostAddr(InetAddress.java:867)
	at java.net.InetAddress.getAddressesFromNameService(InetAddress.java:1302)
	at java.net.InetAddress$NameServiceAddresses.get(InetAddress.java:815)
	at java.net.InetAddress.getAllByName0(InetAddress.java:1291)
	at java.net.InetAddress.getAllByName(InetAddress.java:1144)
	at java.net.InetAddress.getAllByName(InetAddress.java:1065)
	at java.net.InetAddress.getByName(InetAddress.java:1015)
	at com.datastax.spark.connector.cql.CassandraConnectorConf$.maybeResolveHostAndPort(CassandraConnectorConf.scala:346)
	at com.datastax.spark.connector.cql.CassandraConnectorConf$.$anonfun$getIpBasedContactInfoFromSparkConf$1(CassandraConnectorConf.scala:379)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.

23/11/22 17:41:06 WARN ChannelPool: [s1|/172.22.0.2:9042]  Error while opening new channel (ConnectionInitException: [s1|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=6eb8d507-0d60-4560-88e6-4f87b11640f6, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700674778941}): failed to send request (java.nio.channels.NotYetConnectedException))
23/11/22 17:41:06 ERROR CassandraConnectorConf: Unknown host 'p6-db-2'
java.net.UnknownHostException: p6-db-2
	at java.net.InetAddress$CachedAddresses.get(InetAddress.java:764)
	at java.net.InetAddress.getAllByName0(InetAddress.java:1291)
	at java.net.InetAddress.getAllByName(InetAddress.java:1144)
	at java.net.InetAddress.getAllByName(InetAddress.java:1065)
	at java.net.InetAddress.getByName(InetAddress.java:1015)
	at com.datastax.spark.connector.cql.CassandraConnectorConf$.maybeResolveHostAndPort(CassandraConnectorConf.scal

{}

In [25]:
#q8

In [26]:
!docker exec -it p6-db-1 nodetool status

/usr/bin/sh: 1: docker: not found


In [27]:
#q9

In [28]:
# Create a gRPC channel and stub
channel = grpc.insecure_channel('localhost:5440')
stub = station_pb2_grpc.StationStub(channel)

# Make a StationMax request
response = stub.StationMax(station_pb2.StationMaxRequest(id='USW00014837'))

response.error

ValueError: Protocol message StationMaxRequest has no "id" field.

In [29]:
#q10

In [30]:
channel = grpc.insecure_channel('localhost:5440')

# Create a stub (client)
stub = station_pb2_grpc.StationStub(channel)

# Create a valid request message
record_temps_request = station_pb2.RecordTempsRequest(
    id='USW00014837',
    records=[
        station_pb2.TemperatureRecord(date='2023-11-21', tmin=10, tmax=20),
        station_pb2.TemperatureRecord(date='2023-11-22', tmin=15, tmax=25),
    ]
)

# Make the call
response = stub.RecordTemps(record_temps_request)

response.error


AttributeError: module 'station_pb2' has no attribute 'TemperatureRecord'